In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load Data

In [2]:
mnist = read_data_sets("mnist/", one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


# Simple Structure

In [3]:
def simple_fc(inputs):
    net = slim.fully_connected(inputs, 512, scope='FC1')
    net = slim.fully_connected(net, 256, scope='FC2')
    logits = slim.fully_connected(net, 10, activation_fn=None, scope='Logits')
    predictions = slim.softmax(logits)
    return logits, predictions

# configure

In [4]:
gpu_num = 2 
batch_size =200 

# Model

In [5]:
class Model():
    def __init__(self):
        self._create_placeholders()
        self._feed_data()
        with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
            self._create_train_model()
            self._create_test_model()

    def _create_placeholders(self):
        self.inputs = tf.placeholder(
            tf.float32, shape=(None, 784), name='inputs')
        self.labels = tf.placeholder(
            tf.float32, shape=(None, 10), name='labels')

    def _feed_data(self):
        self.X = tf.split(self.inputs, gpu_num)
        self.Y = tf.split(self.labels, gpu_num)

    def _create_train_model(self):
        for gpu_id in range(gpu_num):
            with tf.device(
                    tf.DeviceSpec(device_type="GPU", device_index=gpu_id)):
                print('tower:{}...'.format(gpu_id))
                with tf.variable_scope(
                        tf.get_variable_scope(), reuse=(gpu_id > 0)):
                    logits, predictions = simple_fc(self.X[gpu_id])
                    tf.get_variable_scope().reuse_variables()
                    tower_loss = tf.losses.softmax_cross_entropy(
                        onehot_labels=self.Y[gpu_id], logits=logits)
                    tf.add_to_collection("train_total_loss", tower_loss)
                    train_tower_classes = tf.argmax(input=predictions, axis=1)
                    tf.add_to_collection("train_total_classes",
                                         train_tower_classes)
                    train_tower_labels = tf.argmax(
                        input=self.Y[gpu_id], axis=1)
                    train_tower_accuracy = tf.reduce_mean(
                        tf.cast(
                            tf.equal(train_tower_labels, train_tower_classes),
                            tf.float32))
                    tf.add_to_collection("train_total_accuracy",
                                         train_tower_accuracy)
        print('build train model on gpu tower done.')
        train_loss = tf.reduce_mean(tf.get_collection("train_total_loss"), 0)
        train_op = tf.train.AdamOptimizer(0.001).minimize(
            train_loss, colocate_gradients_with_ops=True)
        train_accuracy = tf.reduce_mean(
            tf.get_collection("train_total_accuracy"), 0)
        train_classes = tf.reshape(
            tf.get_collection('train_total_classes'), [-1, 1])
        self.train_op = train_op
        self.train_loss = train_loss
        self.train_accuracy = train_accuracy
        self.train_classes = train_classes

    def _create_test_model(self):
        for gpu_id in range(gpu_num):
            with tf.device(
                    tf.DeviceSpec(device_type="GPU", device_index=gpu_id)):
                print('test tower:{}...'.format(gpu_id))
                with tf.variable_scope(
                        tf.get_variable_scope(), reuse=(gpu_id > 0)):
                    logits, predictions = simple_fc(self.X[gpu_id])
                    tf.get_variable_scope().reuse_variables()
                    tower_loss = tf.losses.softmax_cross_entropy(
                        onehot_labels=self.Y[gpu_id], logits=logits)
                    tf.add_to_collection("test_total_loss", tower_loss)
                    test_tower_classes = tf.argmax(input=predictions, axis=1)
                    tf.add_to_collection("test_total_classes",
                                         test_tower_classes)
                    test_tower_labels = tf.argmax(input=self.Y[gpu_id], axis=1)
                    test_tower_accuracy = tf.reduce_mean(
                        tf.cast(
                            tf.equal(test_tower_labels, test_tower_classes),
                            tf.float32))
                    tf.add_to_collection("test_total_accuracy",
                                         test_tower_accuracy)
        print('build test model on gpu tower done.')
        test_loss = tf.reduce_mean(tf.get_collection("test_total_loss"), 0)
        test_accuracy = tf.reduce_mean(
            tf.get_collection("test_total_accuracy"), 0)
        test_classes = tf.reshape(
            tf.get_collection('test_total_classes'), [-1, 1])
        self.test_loss = test_loss
        self.test_accuracy = test_accuracy
        self.test_classes = test_classes

# Run Main

In [6]:
model = Model()
config_gpu = tf.ConfigProto()
config_gpu.gpu_options.allow_growth = True

if not tf.gfile.Exists('saved_models'):
    print('create saved_model directory')
    tf.gfile.MkDir('saved_models')


def get_save_vars(exclusions):
    variables_to_save = []
    for var in tf.global_variables():
        excluded = False
        for exclusion in exclusions:
            if exclusion in var.op.name:
                excluded = True
                break
        if not excluded:
            variables_to_save.append(var)
    return variables_to_save


with tf.Session(config=config_gpu) as sess:
    init_op = tf.group(tf.global_variables_initializer(),
                       tf.local_variables_initializer())
    sess.run(init_op)
    saver = tf.train.Saver(get_save_vars(['/Adam']))
    for i in tf.global_variables():
        print(i)
    for epoch in range(100):
        for batch in range(mnist.train.num_examples // batch_size):
            batch = mnist.train.next_batch(batch_size)
            _, train_acc, train_loss = sess.run(
                [model.train_op, model.train_accuracy, model.train_loss],
                feed_dict={
                    model.inputs: batch[0],
                    model.labels: batch[1]
                })
        for batch in range(mnist.test.num_examples // batch_size):
            batch = mnist.test.next_batch(batch_size)
            test_acc, test_loss = sess.run(
                [model.test_accuracy, model.test_loss],
                feed_dict={
                    model.inputs: batch[0],
                    model.labels: batch[1]
                })
        if epoch % 5 == 0:
            saver.save(sess, 'saved_models/{}.ckpt'.format(epoch))
            print(
                "epoch {}, train_acc: {:.5f}, train_loss: {:.5f}, test_acc: {:.5f}, test_loss: {:.5f}".
                format(epoch, train_acc, train_loss, test_acc, test_loss))

tower:0...
tower:1...
build train model on gpu tower done.
test tower:0...
test tower:1...
build test model on gpu tower done.
<tf.Variable 'FC1/weights:0' shape=(784, 512) dtype=float32_ref>
<tf.Variable 'FC1/biases:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'FC2/weights:0' shape=(512, 256) dtype=float32_ref>
<tf.Variable 'FC2/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'Logits/weights:0' shape=(256, 10) dtype=float32_ref>
<tf.Variable 'Logits/biases:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>
<tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>
<tf.Variable 'FC1/weights/Adam:0' shape=(784, 512) dtype=float32_ref>
<tf.Variable 'FC1/weights/Adam_1:0' shape=(784, 512) dtype=float32_ref>
<tf.Variable 'FC1/biases/Adam:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'FC1/biases/Adam_1:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'FC2/weights/Adam:0' shape=(512, 256) dtype=float32_ref>
<tf.Variable 'FC2/weights/Adam_1:0' 